# Topic modelling of news headlines for prediction of news category with 7 topics¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pyLDAvis.lda_model

## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Extract features from 'title_description_text' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_description_text_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<67513x29998 sparse matrix of type '<class 'numpy.int64'>'
	with 1026097 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

29998

In [14]:
cv.get_feature_names_out()

array(['000', '01', '034', ..., 'đoković', 'войны', 'украинцы'],
      dtype=object)

In [15]:
len(lda.components_)

7

In [16]:
lda.components_

array([[ 1.11319917,  0.14315524, 22.10247387, ...,  0.14337529,
         0.14285816,  0.1428576 ],
       [34.31009412,  0.1431382 ,  0.17817771, ...,  0.14619281,
         0.14285808,  0.14285757],
       [ 0.14343256,  4.13878023, 18.17614036, ...,  0.16194595,
         3.1428522 ,  0.14285744],
       ...,
       [ 3.93409388,  0.14285728, 10.57973483, ...,  0.1428577 ,
         0.14285784,  0.14285746],
       [ 3.83678091,  0.14338078, 23.77811105, ...,  3.11949335,
         0.14285769,  0.14285739],
       [ 5.02983794,  0.14285729, 14.73605078, ...,  0.14285775,
         0.14285791,  5.14285496]])

In [17]:
len(lda.components_[0])

29998

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 1589, 12143, 17489, ..., 25501, 19899, 21156], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

online
usa
moskau
un
russisch
china
krieg
ukraine
putin
russland


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['präsident', 'kreml', 'westen', 'baerbock', 'focus', 'online', 'usa', 'moskau', 'un', 'russisch', 'china', 'krieg', 'ukraine', 'putin', 'russland']


Die TOP-15 Wörter für das Thema #1
['team', 'spiel', 'sieg', 'erster', 'dfb', 'partei', 'fc', 'bundesliga', 'frau', 'gewinnen', 'deutsch', 'fußball', 'bayern', 'wm', 'afd']


Die TOP-15 Wörter für das Thema #2
['heute', 'stehen', 'deutschland', 'groß', 'cdu', 'geben', 'wahl', 'kanzler', 'gut', 'spd', 'deutsch', 'online', 'focus', 'scholz', 'berlin']


Die TOP-15 Wörter für das Thema #3
['präsident', 'gipfel', 'zahl', 'generation', 'usa', 'land', 'steigen', 'biden', 'deutschland', 'pandemie', 'eu', 'geben', 'türkei', 'us', 'corona']


Die TOP-15 Wörter für das Thema #4
['geben', 'weit', 'tote', 'iran', 'regierung', 'sterben', 'gericht', 'ex', 'usa', 'polizei', 'protest', 'trump', 'mensch', 'präsident', 'us']


Die TOP-15 Wörter für das Thema #5
['gut', 'fdp', 'groß', 'bundesregierung', 'grüne', 'land', '

### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [24]:
topic_results[0].round(5)

array([0.00623, 0.00622, 0.10575, 0.08399, 0.00622, 0.78537, 0.00622])

In [25]:
topic_results[0].argmax()

5

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...,5
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...,6
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...,2
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...,6
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,3


## Visualize model

In [28]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5      42.266525  12.537951       1        1  19.880240
2       7.058697 -26.207924       2        1  16.845583
4      -9.065055  23.702736       3        1  15.689041
6      13.434557   3.408497       4        1  14.381416
3     -15.373838  -5.891829       5        1  11.314604
1      35.865017 -17.072254       6        1  11.234971
0      19.795969  32.939171       7        1  10.654145, topic_info=              Term          Freq         Total Category  logprob  loglift
25501      ukraine  11791.000000  11791.000000  Default  30.0000  30.0000
21156     russland   5988.000000   5988.000000  Default  29.0000  29.0000
14353        krieg   5936.000000   5936.000000  Default  28.0000  28.0000
21150     russisch   5644.000000   5644.000000  Default  27.0000  27.0000
19899        putin   3945.000000   3945.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
19786    präsident    493.923780   4153.310111   Topic7  -5.5272   0.1099
9307         geben    447.639910   5105.471118   Topic7  -5.6256  -0.1949
5532   deutschland    443.709312   6523.948434   Topic7  -5.6344  -0.4488
29064       zeigen    380.743247   2582.251795   Topic7  -5.7874   0.3249
10573         groß    373.218337   3644.395787   Topic7  -5.8074  -0.0396

[495 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
194        7  0.993835          abe
596        7  0.980167         adac
627        1  0.002074          afd
627        6  0.996935          afd
643        5  0.999090  afghanistan
...      ...       ...          ...
29792      2  0.048140    überblick
29792      4  0.617066    überblick
29792      5  0.059081    überblick
29792      6  0.084245    überblick
29792      7  0.001094    überblick

[1053 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 5, 7, 4, 2, 1])

## Save train set with topics and dump countvectorizer and model

In [29]:
#Save result to csv
train.to_csv('train_lda_combined_7.csv')

In [30]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv_combined.jl')

['models/cv_combined.jl']

In [31]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_7_topics_combined.jl')

['models/lda_model_7_topics_combined.jl']